<a href="https://colab.research.google.com/github/vgarcialopezm/ABC-SMC/blob/main/LV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm, uniform, multivariate_normal
from scipy.optimize import minimize
from scipy.special import logsumexp
import sys,ast
from random import choices,seed,random
from tqdm import tqdm
#import p_tqdm
from functools import partial
import os
import matplotlib.pyplot as plt


epsilons=[30.0,16.0,6.0,4.3,3.5,2.1,1.2,0.8,0.2,0.08]
#epsilons=[16.0,12.0,11.8,11.3,10.6,7.1,5.2]

parametros=[1,1,1,1]
print(type(parametros))

params_lotka_volterra = [ # list of parameters used in the lotka volterra model 
    {'name' : 'a', 'lower_limit':0.0,'upper_limit':10.0},# growing rate of prey in absence of repressor
    {'name' : 'b','lower_limit':0.0,'upper_limit':10.0},#susceptibility of prey 
    {'name' : 'c','lower_limit':0.0,'upper_limit':10.0},# extinction rate of predator
    {'name' : 'd','lower_limit':0.0,'upper_limit':10.0}# benefit of predator
]

In [ ]:
def euc_dist(data1, data2):
    if np.shape(data1) != np.shape(data2):
        print ("\n the dimensions of the datasets are different (%s v.s. %s)\n" % (len(data1), len(data2)))
        sys.exit()
    else:
        distance = np.linalg.norm(data1 - data2)
        print('dist',data1 - data2)

    if distance < 0:
        return [None]
    else:
        return distance


In [ ]:

def euc_disti(data1, data2):
    if np.shape(data1) != np.shape(data2):
        print ("\n the dimensions of the datasets are different (%s v.s. %s)\n" % (len(data1), len(data2)))
        sys.exit()
    else:
        z =np.array((data1[:,0] - data2[:,0])**2+ (data1[:,1] - data2[:,1])**2)
        #print (z)
        distance=np.sum(z)

    if distance < 0:
        return [None]
    elif np.isnan(distance):
      distance=100000
      return distance
    else:
        return distance



In [ ]:
def prior():
### Generate a random parameter inside the limits stablished. The shape of the distribution can be changed if required
    prior = []
    for ipar,par in enumerate(params_lotka_volterra):
        prior.append(uniform.rvs(loc = par['lower_limit'],
                                 scale = par['upper_limit'])) #par['upper_limit']))
        
       
    return prior


In [ ]:
#function that given the values of the parameters, calculates the 

def evaluate_prev_pru(params):
    print('parameters',params)
    l=len(params)
    prior = 1
    for ipar,par in enumerate(params_lotka_volterra):
    #for i in range(l):
        prior *= uniform.pdf(params[ipar],loc = par['lower_limit'],
                                 scale = par['upper_limit'])
        if prior==0:
            break   
      #  print('params i', params[i])
       # print('prior',prior)
    return prior


In [ ]:
#function that, given a list of parameters sampled, perturbs it by applying a multivariate normal kernel
#function that, given a list of parameters sampled, perturbs it by applying a multivariate normal kernel
def perturbi(listaprev,s):
    #print(listaprev)
    lista=np.asarray(listaprev) #.tolist()
    #mean_vec=np.mean(lista)
    k=uniform.rvs(loc = -0.1,scale = 0.2)
    cov_matrix=2.0*np.cov(lista.T)  #the covariance matrix for the multivariate normal perturbation kernel is given by this expression
    kernel=multivariate_normal(cov=cov_matrix)
    pert=s+k # here we obtain the list of perturbed parameters
    pertur=pert.tolist()
    return pertur


In [ ]:
def perturb(listaprev,s):
    #print(listaprev)
    lista=np.asarray(listaprev) #.tolist()
    #mean_vec=np.mean(lista)
    cov_matrix=2.0*np.cov(lista.T)  #the covariance matrix for the multivariate normal perturbation kernel is given by this expression
    kernel=multivariate_normal(cov=cov_matrix)
    pert=s+kernel.rvs() # here we obtain the list of perturbed parameters
    pertur=pert.tolist()
    return pertur


In [ ]:

def rk4(f,in_c,t,params):
    #params=[a,b,c,d]
    #h=t[1]-t[0]
    n=len(t)
    X  = np.zeros([n,len(in_c)],dtype=np.float64)
    X[0]=in_c
    for i in range(n-1):
      h=t[i+1]-t[i]
      k1=f(t[i],X[i],*params)
      k2=f(t[i]+h/2.,X[i]+k1*h/2.,*params)
      k3=f(t[i]+h/2,X[i]+k2*h/2.,*params)
      k4=f(t[i]+h,X[i]+k3*h,*params)
    
      X[i+1]=X[i]+h*(k1/6.+k2/3.+k3/3.+k4/6.)
     
    return X

In [ ]:
def lotka_volterra(t,X,a,b,c,d):
    x,y=X
    dx=a*x-b*x*y
    dy=c*x*y-d*y
    return np.array([dx,dy],dtype=np.float64)

X0=[1,0.5]
#t=np.linspace()
t=[1.1, 2.4, 3.9, 5.6, 7.5, 9.6, 11.9, 14.4]
t1=np.linspace(0,15,1000)

In [ ]:
data1_c=np.array([[1.87, 0.65, 0.22, 0.31, 1.64, 1.15, 0.24, 2.91],[0.49, 2.62, 1.54, 0.02, 1.14, 1.68, 1.07, 0.88]]).T

def add_noise(mu,sigma,data):
    noise=np.random.normal(mu,sigma)
    data_noise=data+noise
    return data_noise
print('\n')
data1_noise=add_noise(0,0.5,midata)
print('data1 noise',data1_noise)


In [ ]:
data1=rk4(lotka_volterra,X0,t1,parametros)
data1

In [ ]:
plt.figure()
plt.plot( t1, data1[:,0],label='predator')
plt.plot( t1, data1[:,1],label='prey')
plt.legend()
plt.xlabel('time')
plt.ylabel('populations density')
#plt.plot(t,data1_c[:,0],'o')
#plt.plot(t,data1_c[:,1],'*')
#plt.plot( t, data3,'x')

In [ ]:
midata=rk4(lotka_volterra,X0,t,parametros)
midata
plt.figure()
plt.plot( t, midata)

In [ ]:
euc_dist(midata,data1_noise)

In [ ]:



#function that gives the denominator used to calculate the weights of every particle.
def weighting(i,j,N,sam,wei,sampre):
     denom=0
     #ker=1
     samprev=np.asarray(sampre)
     cov_matrix=2.0*np.cov(samprev.T)
     kernel=multivariate_normal(cov=cov_matrix)
     for k in range(N):
            #print('sample i j',type(sam[k]),sam[k])
           # print('sample i-1,j',type(sampre[k]),sampre[k])
            sampre[k]=np.array(sampre[k])
            #print('sampre',sampre[k])
            #cov_matrix=2.0*np.cov((sampre[k]).T)  #the covariance matrix for the multivariate normal perturbation kernel is given by this expression
            #print('cov',cov_matrix)
            #kernel=multivariate_normal(cov=cov_matrix)
            # print('wei',wei[i-1,k])
            #print('sam[j]',sam[j])
            #print('sampre[k]',sampre[k])
            ker=kernel.pdf(sam[j]-sampre[k])
            #print('ker',ker)
            #kerne=np.prod(ker)  #here we are obtaining the joint probability of the parameter vector obtained when applying the kernel
            denom+=wei[k]*ker #kerne
            #print('kernel',kernel.cdf(sam[k]-sampre[k]))
     #print('den',denom)      
     return denom


    


In [ ]:
#function used to normalize the weights
def normalize(wei):
    #normalized=wei/np.linalg.norm(wei)
    normalized=wei/np.sum(wei)
    return normalized  



In [ ]:
def principal(epsilons,listaparametros,N,data1,t):
   # accepted_distances = np.loadtxt('smc/distances_{}_{}_{}_{}.out'.format(model,sto,gamma,prior_label))
    T=len(epsilons)
    weight=np.zeros((T,N),float)
    dist=np.zeros((T,N),float)
    sample=np.zeros((T,N),list)
    X0=[1.0,0.5]
    #t=np.linspace(0.,10,10)
    for i in range(T):
        count=0
        counti=0
        label=i
        print("SMC step with target distance: {}".format(epsilons[i]))
        if i==0:
            for j in range (N):
                dist[i,j]=epsilons[i]+1
                while dist[i,j]>epsilons[i]:
                    sample[i,j]=prior()
                    #sample[i,j]=np.array(prior())
                    sample[i,j]=np.asarray(sample[i,j])
                    data2= rk4(lotka_volterra,X0,t,sample[i,j])
                    #print('data2',data2)
                    #data2=np.array(data2, dtype=np.float64)
                    dist[i,j]=euc_disti(data1,data2)
                    #print('distcondata2',dist[i,j])
                count+=1
                print(count)
       
        else:
        
            for j in range (N):
                dist[i,j]=epsilons[i]+1
                while dist[i,j]>epsilons[i]:
                    seed()
                    np.random.seed()
                    choose = choices(sample[i-1,:], weights = weight[i-1,:],k=1)[0] # select a point from the previous sample
                    sample[i,j]=choose
                    #print("before perturb",type(sample[i,j]))
                    #print("before perturb",list(sample[i-1,:]))
                    sample[i,j] = perturb(list(sample[i-1,:]),sample[i,j]) # and perturb it
                    #print("after perturb", sample[i,j])
                    #print("after perturb", type(sample[i,j]))
                    evaluation=evaluate_prev_pru(sample[i,j]) 
                    if evaluation>0:
                        data2=rk4(lotka_volterra,X0,t,sample[i,j])
                        data2=np.array(data2)
                        #print('data2',data2)
                        dist[i,j]=euc_disti(data1,data2)
                        print('distendata2',dist[i,j])
                counti+=1
                print(counti)
        for j in range(N):
            if i==0:
                weight[i,j]=1
               # print(weight[i,j])
            else:
                denom=weighting(i,j,N,sample[i,:],weight[i-1,:],list(sample[i-1,:]))
                weight[i,j]=evaluate_prev_pru(sample[i,j])/denom
        #print('weight[i,:]',weight[i,:])
        if i!=0:
           weight[i,:]=normalize(weight[i,:])
           #print('weight[i,:] normalized',weight[i,:])
        #pars = np.loadtxt('smc_van/pars_{}.out'.format(i))
        #weights = np.loadtxt('smc_van/weights_{}.out'.format(i))
        #np.savetxt('smc_van/pars_{}.out'.format(i), sample[T-1,:])
        #np.savetxt('smc_van/weights_{}.out'.format(i), weight[T-1,:])
      #  np.savetxt('smc/distances_{}.out'.format(label), accepted_distances)
    #print('sample',sample[T-1,N-1])
    #print('weight',weight[T-1])
    #print('dist',dist[T-1])
    return sample, weight, dist,data2


In [ ]:
sample,weight,dist,data2=principal(epsilons,params_lotka_volterra,100,midata,t)

In [ ]:

print("min accepted distance: ",np.min(dist[-1,:]))
print("median accepted distance: ",np.median(dist[-1,:]))
#print("median evaluated distance: ",np.median(evaluated_distances))


In [ ]:
#np.median(sample[-1,:])
np.var(sample[-1,0])
sample[-1,:]

In [ ]:
parama=[]
for j in sample[-1,:]:
    parama.append(j[0])
    


In [ ]:
amean=np.mean(parama)
amed=np.median(parama)
avar=np.var(parama)
print('mean',amean)
print('median',amed)
print('variance',avar)

In [ ]:
paramb=[]
for j in sample[-1,:]:
    paramb.append(j[1])
    

In [ ]:
bmean=np.mean(paramb)
bmed=np.median(paramb)
bvar=np.var(paramb)
print('mean',bmean)
print('median',bmed)
print('variance',bvar)

In [ ]:
paramc=[]
for j in sample[-1,:]:
    paramc.append(j[2])
    


In [ ]:
cmean=np.mean(paramc)
cmed=np.median(paramc)
cvar=np.var(paramc)
print('mean',cmean)
print('median',cmed)
print('variance',cvar)

In [ ]:
paramd=[]
for j in sample[-1,:]:
    paramd.append(j[3])
    

In [ ]:
dmean=np.mean(paramd)
dmed=np.median(paramd)
dvar=np.var(paramd)
print('mean',dmean)
print('median',dmed)
print('variance',dvar)

In [ ]:
plt.figure()
plt.plot(t,midata,'o')
for i in sample[-1,:]:
    data=rk4(lotka_volterra,X0,t,i)
    plt.plot(t,data)
plt.show()

In [ ]:
for k in range(len(epsilons)):
    for j in range(len(params_lotka_volterra)):
        plt.figure()
        b=[]
        for i in sample[k,:]:
            b.append(i[j])
        al=np.histogram(b)
        n,bins,patches=plt.hist(b,edgecolor='white')
        plt.title('Histogram of parameter'+str(j))
    plt.show()
plt.show()


In [ ]:
print("min accepted distance: ",np.min(dist))

In [ ]:
mindis=np.min(dist[-1,:])
np.where(dist==mindis)

In [ ]:
maxdis=np.max(dist[-1,:])
np.where(dist==maxdis)

In [ ]:
mindis

In [ ]:
dist[-1,:]

In [ ]:
data4=rk4(lotka_volterra,X0,t,sample[-1,94])

In [ ]:
plt.figure()
plt.plot( t1, data1[:,0],label='predator')
plt.plot( t1, data1[:,1],label='prey')
plt.legend()
plt.xlabel('time')
plt.ylabel('populations density')
#plt.plot(t,data1_c[:,0],'o')
#plt.plot(t,data1_c[:,1],'*')
#plt.plot( t, data3,'x')

In [ ]:
plt.figure()
plt.plot()
plt.plot(t,midata,'o-',label='target data')
plt.plot(t,data4,'*',label='approximation')
plt.legend(loc='upper right')
plt.xlabel('time')
plt.ylabel('populations density')
plt.show()